### Get Two Nearest Features for Each Grid Centroid
This notebook is the second notebook in the `Phase 3: Transmission Modelling` workflow. We iterate through each grid dataset, and each transmission line dataset, and generate a 'Near Table' containing the coordinates, distance, and feature ID for the closest two transmission lines to each grid cell centroid. The relationships established in this notebook become the basis for the least cost path analysis in part 3.  

In [1]:
import arcpy 
import os
from arcpy import env
from arcpy.sa import *

#### Define Useful Functions

In [ ]:
def getCentroid(grid_fc, cpa_fc = wind_area): # we will only use wind cpa because that excludes less land

    # Select all grid cells that overlap with our CPAs
    grid_cells = arcpy.management.SelectLayerByLocation(grid_fc, 'INTERSECT', 
                                                          cpa_fc, 0, 
                                                          'NEW_SELECTION') 

    # Get name of input grid
    grid_name = os.path.splitext(os.path.basename(grid_fc))[0]

    # Get name of area
    area_name = os.path.splitext(os.path.basename(cpa_fc))[0]

    # Set ouput feature class name
    output_name = "{}_centroid".format(grid_name) 
    output_path = os.path.join(centroid_workspace, output_name)

    # Convert polygon feature class to point feature class, with the point representing the CENTROID of the feature
    centroid_fc = arcpy.management.FeatureToPoint(grid_cells, output_path, "CENTROID")

    print(f'{output_path}')
    print("centroid saved to {}".format(output_path))

In [ ]:
def getFeatureClasses(input_workspace):
    previous_workspace = arcpy.env.workspace
    try:
        arcpy.env.workspace = input_workspace
        feature_class_list = arcpy.ListFeatureClasses()
        return feature_class_list
    finally:
        arcpy.env.workspace = previous_workspace

#### Set Input and Output Paths

In [2]:
# Paths to grids 
cesm2_grid = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\grids.gdb\\cesm2_base_grid"
era5_grid = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\grids.gdb\\era5_base_grid"
solar_area = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\resourceAssessment.gdb\\InclusionArea_Solar_SL1_ZRM_filters_dissolved"
wind_area = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\data\\resourceAssessment.gdb\\InclusionArea_Wind_SL1_ZRM_filters_dissolved"

# Set output workspace and environment
mainOutputFolder = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data"
centroid_workspace = os.path.join(mainOutputFolder, "scratch.gdb")
arcpy.env.workspace = centroid_workspace
arcpy.env.overwriteOutput = True

#### Create Grid Centroid

In [ ]:
# Get list for grids and areas
grid_list = [cesm2_grid]
area_list = [wind_area]

In [ ]:
# Get the centroid 
for grid in grid_list:
    for area in area_list:
        getCentroid(grid, area)

In [ ]:
# Look at the fields of the created feature class
sample_fc = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\scratch.gdb\\cesm2_base_grid_centroid"
fields = [field.name for field in arcpy.ListFields(sample_fc)]
with arcpy.da.SearchCursor(sample_fc, fields) as cursor:
    for row in cursor:
        print(row)

In [8]:
# set path to input workspace for transmission lines
transmission_lines = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\TransmissionLines.gdb"


In [ ]:
voltage_class_list = getFeatureClasses(transmission_lines)
voltage_class_list

In [ ]:
# Get centroid points 
input_grid_list = arcpy.ListFeatureClasses('*cesm2_base_grid_centroid')
input_grid_list

#### Generate Near Table Analysis

In [ ]:
for input_grid in input_grid_list:
    for voltage_class in voltage_class_list:

        # Get the full path for the voltage path
        voltage_class_path = os.path.join(transmission_lines, voltage_class)

        # Set output feature class name
        near_name = "{}_{}_near_analysis_twogeos".format(input_grid, voltage_class)
        near_path = os.path.join(centroid_workspace, near_name)

        # Copy input grid
        arcpy.CopyFeatures_management(input_grid, near_path)

        try:
            # Perform near analysis on copied input grid to find the two nearest geometries
            near_table = arcpy.GenerateNearTable_analysis(near_path, voltage_class_path, 'in_memory\\near_table', search_radius = "", location = "LOCATION", angle = "NO_ANGLE", closest = "ALL", closest_count = 2)

            # Extract NEAR_X, NEAR_Y, and NEAR_FID from the near table
            with arcpy.da.SearchCursor(near_table, ['IN_FID', 'NEAR_X', 'NEAR_Y', 'NEAR_FID', 'NEAR_DIST', 'NEAR_RANK']) as cursor:
                for row in cursor:
                    in_fid = row[0]
                    near_x = row[1]
                    near_y = row[2]
                    near_fid_value = row[3]
                    near_dist = row[4]
                    near_rank = row[5]

                    # Update the point feature class with NEAR_X, NEAR_Y, and NEAR_FID for the current NEAR_RANK
                    update_fields = [f'NEAR_X_{near_rank}', f'NEAR_Y_{near_rank}', f'NEAR_FID_{near_rank}', f'NEAR_DIST_{near_rank}']

                    # Check if the fields exist, and if not, create them
                    existing_fields = [field.name for field in arcpy.ListFields(near_path)]
                    for field_name in update_fields:
                        if field_name not in existing_fields:
                            arcpy.AddField_management(near_path, field_name, "DOUBLE")
                
                    with arcpy.da.UpdateCursor(near_path, update_fields, f'OBJECTID = {in_fid}') as update_cursor:
                        for update_row in update_cursor:
                            update_row[0] = near_x
                            update_row[1] = near_y
                            update_row[2] = near_fid_value
                            update_row[3] = near_dist
                            update_cursor.updateRow(update_row)
            
            # Clean up the near table and line layer
            arcpy.Delete_management(near_table)

            # get geoprocessing messages
            print(arcpy.GetMessages())

        except arcpy.ExecuteError:
            print(arcpy.GetMessages(2))
        except Exception as err:
            print(err.args[0])